In [0]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [0]:
dbutils.widgets.text('catalogo_source','catalog_dev')
dbutils.widgets.text('schema_source','bronze')
dbutils.widgets.text('schema_sink','silver')

In [0]:
catalogo = dbutils.widgets.get('catalogo_source')
schema_source = dbutils.widgets.get('schema_source')
schema_sink = dbutils.widgets.get('schema_sink')

Departamentos

In [0]:
# lectura de la data
df_Departamentos = spark.table(f"{catalogo}.{schema_source}.TblDepartamentos")


In [0]:
#creando vista temporal para transformacion
# En la transformacion se validan valores Null

df_Departamentos.createOrReplaceTempView("Departamentos_data")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW VW_Departamentos AS (

Select 
IdDepartamento,
UPPER(NombreDepartamento) as NombreDepartamento
From Departamentos_data

)

In [0]:
#Almacenado el resultado de nuestra query en un dataframe

query = """
Select
IdDepartamento,
NombreDepartamento
From VW_Departamentos
"""

df_VU = spark.sql(query)

In [0]:
#almacenando nuestra transformacion en nuestra capa silver
df_VU.write.mode('overwrite').format('delta').saveAsTable(f'{catalogo}.{schema_sink}.TblDepartamentos')

PuestosTrabajo

In [0]:
# lectura de la data de PuestosTrabajo
df_PuestosTrabajo = spark.table(f"{catalogo}.{schema_source}.TblPuestosTrabajo")

In [0]:
#creando vista temporal para transformacion
# En la transformacion se validan valores Null

df_PuestosTrabajo.createOrReplaceTempView("PuestosTrabajo_data")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW VW_PuestosTrabajo AS (

Select 
IdPuesto,
UPPER(NombrePuesto) as NombrePuesto, 
IdDepartamento
From PuestosTrabajo_data

)

In [0]:
#Almacenado el resultado de nuestra query en un dataframe

query = """
Select
IdPuesto,
NombrePuesto,
IdDepartamento
From VW_PuestosTrabajo
"""

df_VistaPuesto = spark.sql(query)

In [0]:
#almacenando nuestra transformacion en nuestra capa silver
df_VistaPuesto.write.mode('overwrite').format('delta').saveAsTable(f'{catalogo}.{schema_sink}.TblPuestosTrabajo')

Empleados

In [0]:
# lectura de la data de Empleados
df_Empleados = spark.table(f"{catalogo}.{schema_source}.TblEmpleados")

In [0]:
#creando vista temporal para transformacion
# En la transformacion se validan valores Null

df_Empleados.createOrReplaceTempView("Empleados_data")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW VW_Empleados AS (

Select 
IdEmpleado,
UPPER(NombreEmpleado) As NombreEmpleado,
lower(Correo) As Correo,
Salario,
IdPuesto
From Empleados_data

)

In [0]:
#Almacenado el resultado de nuestra query en un dataframe

query = """
Select
IdEmpleado,
NombreEmpleado,
Correo,
Salario,
IdPuesto
From VW_Empleados
"""

df_VistaEmpleados = spark.sql(query)

In [0]:
#almacenando nuestra transformacion en nuestra capa silver
df_VistaEmpleados.write.mode('overwrite').format('delta').saveAsTable(f'{catalogo}.{schema_sink}.TblEmpleados')